In [18]:
!pip install xgboost==1.6.1
!pip install scikit-optimize


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
#from google.colab import drive
#drive.mount('/content/drive')

choose between xgbr, dtr and ann (1.rfr ,2.xgbr, 3.ann)

In [20]:
#from xgboost import XGBRegressor
#new_model=XGBRegressor()
#new_model.load_model('/content/drive/MyDrive/Colab Notebooks/xgbr_avggc.h5')
#

#pickle for gbr
import pickle
with open('/content/drive/MyDrive/Colab Notebooks/gbr_avggc4in.pkl' , 'rb') as f:
    new_model = pickle.load(f)

#from keras import models
# Recreate the exact same model, including its weights and the optimizer(r2/3)
#new_model = models.load_model('/content/drive/MyDrive/Colab Notebooks/my_tikamaveragegcr5-5r6.h5')
# Show the model architecture
#new_model.summary()

In [21]:
import pandas as pd
import numpy as np
test=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/average GC full dataset new for colab 1007 without powder mass.csv')
Xtest=test.iloc[:,:4].values
Ytest = test.iloc[:,4:].values
print(Xtest[0])


[2.00000000e+00 3.55247982e-01 3.00000000e+00 1.00000000e+03]


In [22]:
#prediction=new_model.predict(Xtest)
#print(prediction)
#print(Ytest)

#import numpy as np

#Defining MAPE function
#def MAPE(Y_actual,Y_Predicted):
 #   mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
  #  return mape

#LR_MAPE= MAPE(Ytest,prediction)
#print("MAPE: ",LR_MAPE)
#print(Ytest[0,2])

In [23]:
#df=pd.DataFrame(prediction,index=None)
#arr=df[2]
#arr2=arr.to_numpy()
#print(arr2)

In [24]:
target=[(15,10,45)]
weight=[(0.5,0.1,1)]

def prediction(input):
   
   pred=new_model.predict(input)
   preddf=pd.DataFrame(pred,index=None)
   predvalue=preddf[2]
   predarr=predvalue.to_numpy()


   return predarr


In [25]:
def fitness(input):
    ans=prediction(input)
    return ans
  


In [26]:
target=[(15,10,45)]
weight=[(0.5,0.1,1)]
import random
#generate solutions
input=[]
for s in range(20):
    NPCtype=random.randint(1,2)
    
    #NPCweight=random.randrange(50000,61000,100)/1000
    loading=random.randrange(25,60,1)/100
    KOHconc=random.choice([1,3,0.75])
    #ratio=random.randrange(7,11,1)
    current=random.randrange(100,1001,100)


    input.append([(NPCtype,loading,KOHconc,current)])

input=np.asarray(input)
mating_pool=input.reshape(input.shape[0],input.shape[2])



In [27]:
test=prediction(mating_pool)
test=np.asarray(test)

print(mating_pool)
print(test)

[[2.0e+00 4.0e-01 7.5e-01 4.0e+02]
 [1.0e+00 5.4e-01 7.5e-01 4.0e+02]
 [1.0e+00 4.0e-01 7.5e-01 5.0e+02]
 [2.0e+00 4.5e-01 7.5e-01 8.0e+02]
 [2.0e+00 3.9e-01 7.5e-01 9.0e+02]
 [2.0e+00 3.1e-01 3.0e+00 5.0e+02]
 [1.0e+00 3.8e-01 1.0e+00 2.0e+02]
 [1.0e+00 2.6e-01 3.0e+00 8.0e+02]
 [2.0e+00 4.1e-01 7.5e-01 9.0e+02]
 [1.0e+00 4.8e-01 7.5e-01 6.0e+02]
 [1.0e+00 4.4e-01 3.0e+00 9.0e+02]
 [1.0e+00 3.5e-01 7.5e-01 3.0e+02]
 [1.0e+00 3.3e-01 7.5e-01 3.0e+02]
 [2.0e+00 4.5e-01 1.0e+00 4.0e+02]
 [1.0e+00 3.0e-01 1.0e+00 7.0e+02]
 [2.0e+00 3.5e-01 7.5e-01 3.0e+02]
 [1.0e+00 3.0e-01 3.0e+00 1.0e+03]
 [2.0e+00 3.0e-01 3.0e+00 9.0e+02]
 [1.0e+00 2.9e-01 7.5e-01 5.0e+02]
 [2.0e+00 4.7e-01 1.0e+00 1.0e+02]]
[39.70961299 34.04233882 39.28714461 35.32402913 42.80824538 31.52985503
 21.40510538 30.16723324 42.17600632 36.21345719 35.73889286 16.58539713
 32.29569122 27.88732228 23.30544613 16.31250865 23.54786145 29.49058252
 13.60449673 15.31241328]


In [28]:
#select first four as parents(best 4)
def parent_selection(mixsort):
    fitness_mating_pool=prediction(mating_pool)
    mix_mating_fitness=np.column_stack((mating_pool,fitness_mating_pool))

    mixsort=mix_mating_fitness[mix_mating_fitness[:, 4].argsort()]
    parents_pool=mixsort[:,0:4]
    parents_results=mixsort[:,4]
    return parents_pool

In [29]:
# crossover(for mating)
def uniform_crossover(A,B,C,D,P):
    for i in range(len(P)):
        if P[i]<0.5:
          temp=A[i]
          A[i]=B[i]
          B[i]=C[i]
          C[i]=D[i]
          D[i]=temp
    return A,B,C,D

def crossover_process(mating_pool):
    A=mating_pool[0]
    B=mating_pool[1]
    C=mating_pool[2]
    D=mating_pool[3]
    P=np.random.rand(4)
    crossover_pool=uniform_crossover(A,B,C,D,P)
    crossover_pool=np.asarray(crossover_pool)
    return crossover_pool


#crossover_pool=crossover_process(parents_pool)
#print(crossover_pool)



In [30]:
#mutation!
def mutation(crossover_pool):
    mutation_rate=random.uniform(0.8,1.2)
    mutation_X=crossover_pool[:,0]
    mutation_pool=crossover_pool[:,1:]*mutation_rate
    final_mutation=np.column_stack((mutation_X,mutation_pool))
    return final_mutation

#mutation_pool=mutation(crossover_pool)
#print(mutation_pool)
#print(mutation_pool)
#mutation_rate=random.uniform(0.98,1.02)
#mutation_pool=crossover_pool*mutation_rate


In [31]:
#survival selection by def
def survivor_selection():
    new_mating_pool=np.insert(mutation(crossover_process(parent_selection(mating_pool))),0,mating_pool,axis=0)
    fitness_new_pool=fitness(new_mating_pool)
    mix_new_fitness=np.column_stack((new_mating_pool,fitness_new_pool))

    mixsortnew=mix_new_fitness[mix_new_fitness[:, 4].argsort()]
    mixsortnew =mixsortnew[::-1]
    mixsortnew=mixsortnew[:]
    new_mating_pool=mixsortnew[0:20,0:4]
    return new_mating_pool


test2=survivor_selection()
print(test2)

[[2.00000000e+00 3.90000000e-01 7.50000000e-01 9.00000000e+02]
 [2.00000000e+00 4.10000000e-01 7.50000000e-01 9.00000000e+02]
 [2.00000000e+00 4.00000000e-01 7.50000000e-01 4.00000000e+02]
 [1.00000000e+00 4.00000000e-01 7.50000000e-01 5.00000000e+02]
 [1.00000000e+00 4.80000000e-01 7.50000000e-01 6.00000000e+02]
 [1.00000000e+00 4.40000000e-01 3.00000000e+00 9.00000000e+02]
 [2.00000000e+00 4.50000000e-01 7.50000000e-01 8.00000000e+02]
 [1.00000000e+00 5.40000000e-01 7.50000000e-01 4.00000000e+02]
 [2.00000000e+00 4.92714511e-01 1.04832875e+00 3.14498624e+02]
 [1.00000000e+00 3.30000000e-01 7.50000000e-01 3.00000000e+02]
 [2.00000000e+00 3.10000000e-01 3.00000000e+00 5.00000000e+02]
 [1.00000000e+00 2.60000000e-01 3.00000000e+00 8.00000000e+02]
 [2.00000000e+00 3.00000000e-01 3.00000000e+00 9.00000000e+02]
 [2.00000000e+00 4.50000000e-01 1.00000000e+00 4.00000000e+02]
 [1.00000000e+00 3.66915061e-01 7.86246560e-01 5.24164373e+02]
 [2.00000000e+00 3.66915061e-01 7.86246560e-01 3.144986

In [32]:
def genetic_algorithm():
    parents=parent_selection(mating_pool)
    
    
    cross_p=crossover_process(parents)
   
    mutant_p=mutation(cross_p)
    survived_p=survivor_selection()
    return survived_p
    


print(genetic_algorithm())


[[2.00000000e+00 3.90000000e-01 7.50000000e-01 9.00000000e+02]
 [2.00000000e+00 4.10000000e-01 7.50000000e-01 9.00000000e+02]
 [2.00000000e+00 4.00000000e-01 7.50000000e-01 4.00000000e+02]
 [1.00000000e+00 4.00000000e-01 7.50000000e-01 5.00000000e+02]
 [1.00000000e+00 4.80000000e-01 7.50000000e-01 6.00000000e+02]
 [1.00000000e+00 4.40000000e-01 3.00000000e+00 9.00000000e+02]
 [2.00000000e+00 4.50000000e-01 7.50000000e-01 8.00000000e+02]
 [1.00000000e+00 5.40000000e-01 7.50000000e-01 4.00000000e+02]
 [1.00000000e+00 3.30000000e-01 7.50000000e-01 3.00000000e+02]
 [2.00000000e+00 3.10000000e-01 3.00000000e+00 5.00000000e+02]
 [1.00000000e+00 2.60000000e-01 3.00000000e+00 8.00000000e+02]
 [1.00000000e+00 3.06417545e-01 7.92459169e-01 5.28306113e+02]
 [2.00000000e+00 3.00000000e-01 3.00000000e+00 9.00000000e+02]
 [2.00000000e+00 4.50000000e-01 1.00000000e+00 4.00000000e+02]
 [2.00000000e+00 3.69814279e-01 7.92459169e-01 3.16983668e+02]
 [1.00000000e+00 3.69814279e-01 7.92459169e-01 3.169836

In [33]:
initial_gen=0
max_gen=1000



while initial_gen<max_gen:
   x_new=genetic_algorithm()
   initial_gen+=1 
   mating_pool=x_new
   
   first=x_new[0]
   first=np.asarray(first)
   print([first])
   print("gen",initial_gen,"=",x_new)
   
   test=new_model.predict(x_new)
   c2test=test[:,2]+test[:,4]
   print("best results:",test[0])
   
   print("Fittest C2H4+ethanol:",c2test[0])
   if c2test[0]>=75 : break
   

Streaming output truncated to the last 5000 lines.
 [1.00000000e+00 3.29929597e-01 1.09976532e+00 7.69835727e+02]
 [1.00000000e+00 4.20717210e-01 3.92954715e+00 6.54924525e+02]
 [2.00000000e+00 3.55074619e-01 7.29516730e-01 7.79432091e+02]
 [2.00000000e+00 3.54922580e-01 7.25452833e-01 7.79098346e+02]
 [2.00000000e+00 3.58660605e-01 7.29204359e-01 7.51015558e+02]
 [2.00000000e+00 3.56757987e-01 6.69901350e-01 7.83127288e+02]
 [2.00000000e+00 3.58803120e-01 7.29494110e-01 7.51313976e+02]]
best results: [14.55691775 12.81188673 46.41141592  1.30769776 20.21193795]
Fittest C2H4+ethanol: 66.62335386693127
[array([2.00000000e+00, 3.35355800e-01, 6.81822613e-01, 7.58194414e+02])]
gen 784 = [[2.00000000e+00 3.35355800e-01 6.81822613e-01 7.58194414e+02]
 [2.00000000e+00 3.34329442e-01 7.06820177e-01 7.71529481e+02]
 [2.00000000e+00 3.21494813e-01 3.11124012e+00 8.29664033e+02]
 [1.00000000e+00 3.77378801e-01 4.71827972e+00 8.02829596e+02]
 [1.00000000e+00 3.93892628e-01 2.73264623e+00 8.707711

In [34]:
print(test[0],mating_pool[0])
print(new_model.predict(mating_pool))


[18.93968348 15.50126356 46.51547464  1.31650284 18.13169814] [1.00000000e+00 3.13648683e-01 2.04610039e+00 7.73165927e+02]
[[18.93968348 15.50126356 46.51547464  1.31650284 18.13169814]
 [14.55691775 12.81188673 46.41141592  1.30769776 20.21193795]
 [14.55691775 12.81188673 46.41141592  1.30769776 20.21193795]
 [15.85113448 16.39532733 45.83882391  1.59848201 19.83990611]
 [13.63161174 23.2133901  43.83491963  1.72168125 15.17879325]
 [13.63161174 23.2133901  43.83491963  1.72168125 15.17879325]
 [13.63161174 23.2133901  43.83491963  1.72168125 15.17879325]
 [13.63161174 23.2133901  43.83491963  1.72168125 15.17879325]
 [23.49281443 16.90901122 43.82409761  1.36551617 16.14876852]
 [23.49281443 16.90901122 43.82409761  1.36551617 16.14876852]
 [14.55691775 23.36022939 43.82409761  1.4818268  15.91443197]
 [23.49281443 16.90901122 43.82409761  1.36551617 16.14876852]
 [23.49281443 16.90901122 43.82409761  1.36551617 16.14876852]
 [14.55691775 23.36022939 43.82409761  1.4818268  15.9144